<a href="https://colab.research.google.com/github/schnap3380/kewwords_for_journal_analysis/blob/main/papers_in_journals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time

In [14]:
# Semantic Scholar API key (by request - mine allows for 100 requests / second)
headers={'x-api-key':'api_key'}
headers.get('x-api-key')

'api_key'

In [12]:
import pandas as pd
import requests
import time

def fetch_articles_from_journal(api_key, journal_name):
    url = "https://api.semanticscholar.org/graph/v1/paper/search/bulk"
    token = None
    all_articles = []

    while True:
      params = {
        "venue": journal_name,
        "publicationTypes": "Review,JournalArticle",
        "year": "2005-2024",
        "fields": "title,abstract,year",
        "limit": 1000,
        "apiKey": api_key
      }
      if token:
              params['token'] = token


      response = requests.get(url, params=params)

      while response.status_code == 429:
        print("Waiting 1 second for access to the API...")
        time.sleep(1)
        response = requests.get(url, params=params)
      if response.status_code != 200:
        raise Exception(f"Error fetching data from Semantic Scholar: {response.status_code}")


      data = response.json()

      if 'data' not in data:
          return pd.DataFrame()  # Return an empty DataFrame if no data is found

    # Process each article and add to the list
      articles = [{
          'Title': article.get('title', ''),
          'Abstract': article.get('abstract', ''),
          'Year': article.get('year', '')
      } for article in data.get('data', [])]
      all_articles.extend(articles)

      token = data.get('token')  # Update the token from the response
      if not token:
        break  # Exit the loop if no next token is provided

    return pd.DataFrame(all_articles)


In [13]:
api_key = 'api_key'
journal_name = 'Nature'
df2 = fetch_articles_from_journal(api_key, journal_name)
df2

,Title,Abstract,Year
0,A dynamic magnetic tension force as the cause ...,None,2015
1,Ms Anaria's classroom rules for well-behaved k...,None,2024
2,PGE2 inhibits TIL expansion by disrupting IL-2...,None,2024
3,China's Chang'e-6 collects first rock samples ...,None,2024
4,Finding the missing heritability of complex di...,None,2009
...,...,...,...
32773,Binning out-of-date chemicals? Somebody think ...,None,2024
32774,Transcript RNA-templated DNA recombination and...,None,2014
32775,Neural population dynamics during reaching,None,2012
32776,Control of chiral orbital currents in a coloss...,None,2022
